In [12]:
import os

from langchain.chains import ConversationChain
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI

In [31]:
OPENAI_API_KEY = "API_KEY"
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
model_name = "gpt-4o-mini-2024-07-18"

In [32]:
template_content = """You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only 
respond once as 'assistant'. 

System Role: Resume Reviewer 

Your role is to act as a resume coach. You will assist users in improving their resumes to better align with specific 
job descriptions. Provide professional advice on resume building, interview preparation, and career development. 
Offer constructive feedback and encouragement. Whenever you are given a resume and a job description, there will be 
tokens added before and after the resume and job description. The tokens are as follows: <RESUME STARTS HERE> and 
<RESUME ENDS HERE> for the resume and <JOB DESCRIPTION STARTS HERE> and <JOB DESCRIPTION ENDS HERE> for the job 
description. Utilize these tokens to provide feedback and suggestions and clearly segregate the resume and job
description. Do not mix up the content of the resume and job description. In case the resume or job requirements 
in the description do not align with each other, do not mix up the content of the resume and job description and 
keep them separate and process them accordingly. Provide feedback based only on the content provided.
"""

resume_analysis_prompt = "Provide a detailed summary of the candidate's skills, experience, and qualifications based on the content of the following resume: <RESUME STARTS HERE> {}. <RESUME ENDS HERE>"
job_description_analysis_prompt = "List the key skills, qualifications, and experience required as outlined in the following job description: <JOB DESCRIPTION STARTS HERE> {}. <JOB DESCRIPTION ENDS HERE>"
comparison_prompt = "Compare the resume: <RESUME STARTS HERE> {}. <RESUME ENDS HERE> with the job description: <JOB DESCRIPTION STARTS HERE> {}.<JOB DESCRIPTION ENDS HERE> Do they match? If not, what are the gaps? Do not make any assumptions about the candidate's skills or experience or the job requirements."
gap_analysis_prompt = "Compare the skills and experience detailed in this resume: <RESUME STARTS HERE> {} <RESUME ENDS HERE> with the requirements listed in the job description: <JOB DESCRIPTION STARTS HERE> {}. <JOB DESCRIPTION ENDS HERE> Identify any gaps or mismatches and list them out specificly. Please provide examples on how to fulfill those gaps."
experience_enhancement_prompt = "Based on the candidate's experience outlined in this resume: <RESUME STARTS HERE> {} <RESUME ENDS HERE>, recommend practical activities or steps to gain or improve the experience aligned with the needs of this role: <JOB DESCRIPTION STARTS HERE> {}. <JOB DESCRIPTION ENDS HERE>. Include recommendations for additional qualifications or certifications."
resume_formatting_prompt = "Offer guidance on how the candidate can enhance the formatting of their resume: <RESUME STARTS HERE> {} <RESUME ENDS HERE> to improve visual appeal and readability."
resume_length_prompt = "Recommend strategies for the candidate to adjust the length of their resume: <RESUME STARTS HERE> {} <RESUME ENDS HERE>, ensuring it is concise while remaining aligned with the requirements in the job description: <JOB DESCRIPTION STARTS HERE> {}. <JOB DESCRIPTION ENDS HERE>"

In [15]:
def generate_llm_response(prompt_input):
    output = resume_chain.predict(input=prompt_input)
    return output

In [16]:
system_message = SystemMessage(content=template_content)
human_message = HumanMessagePromptTemplate.from_template("{history} User:{input} Assistant:")

prompt_template = ChatPromptTemplate(messages=[system_message, human_message], validate_template=True)

memory = ConversationBufferWindowMemory(k=3)

### Chain of Thought(COT) Prompting

![Image of Yaktocat](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fcot.1933d9fe.png&w=1920&q=75)

In [40]:
cot_chain = ConversationChain(
        llm=ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo-1106"),
        verbose=False
)

In [44]:
response = cot_chain.predict(input="""
Your finance department has $23,000 in the budget. 
If they allocate $20,000 for a marketing campaign and add $6,000 from other savings, 
how much is left in the budget?
""")
print(response)

After allocating $20,000 for the marketing campaign and adding $6,000 from other savings, the finance department would have a total of $26,000 in the budget. Therefore, there would be $3,000 left in the budget after these allocations.


In [47]:
response = cot_chain.predict(input="""
System Prompt:

For tasks requiring reasoning or math, use the Chain-of-Thought methodology to explain your step-by-step calculations or logic.

Q: Your finance department has $23,000 in the budget. If they allocate $20,000 for a marketing campaign 
and add $6,000 from other savings, how much is left in the budget?
Let’s think step by step.
""")
print(response)

First, we start with the initial budget of $23,000. Then, we allocate $20,000 for the marketing campaign, leaving us with $3,000. Next, we add $6,000 from other savings to the remaining budget of $3,000, giving us a total budget of $9,000. Therefore, there would be $9,000 left in the budget after these allocations.


<hr>

### Resume Analysis

In [28]:
llm = ChatOpenAI(temperature=0.0, model=model_name)

In [ ]:
resume_chain = ConversationChain(
        llm=llm,
        prompt=prompt_template,
        memory=memory,
        verbose=False
)

In [19]:
resume_content = """
senior data scientist  \nexample by resume genius  \nchicago, il  •  your.email @email.com •  ( 312) 875-3382  
\n \naccomplished senior data scientist with 10+ years of experience driving innovative solutions and delivering 
\nactionable insights. expertise in machine learning, statistical modeling, and big data analytics. 
proven track record of \nleading cross -functional te ams and implementing data -driven strategies that have 
generated $50m+ in revenue.  \nprofessional experience  \n \nsenior data scientist  \nhargrove tech , chicago, 
il  \naugust 20 xx–present  \n lead a team of 8 data scientists in developing and implementing advanced machine 
learning models, resulting \nin a 25% increase in predictive accuracy  \n spearheaded a company -wide initiative
to optimize data pipelines, reducing processing time by 40% and \nsaving $1.2m annually  \n collaborate with
cross -functional teams to develop data -driven strategies, contributing to a 15% increase in \noverall revenue 
\nlead data scientist  \nsuntrust analytics , chicago, il  \njuly 20 xx–august 20 xx \n developed and deployed 
a real -time fraud detection system using ensemble learning techniques, saving the \ncompany $3.5m in potential 
losses  \n built and maintained a customer segmentation model using clustering algorithms, resulting in a 20% 
increase \nin targeted marketing roi  \n conducted regular workshops and training sessions on advanced 
data science techniques, upskilling the team \nand improving overall performance by 30%  \ndata analyst  
\nmaverick solutions, new york, ny  \njuly 20 xx–august 20 xx \n analyzed large datasets using sql and 
python, providing actionable insights to business stakeholders  \n developed and maintained etl processes, 
ensuring timely and accurate data integration from multiple \nsources  \n created ad -hoc reports and analyses, 
contributing to data -driven decision making  \neducation  \n \nnew york university , new york, ny  \nms in data science  \nmay 20xx  \nadditional skills  \n \n● machine learning:  supervised learning, unsupervised learning, deep learning, neural networks, \nensemble methods, natural language processing (nlp)  \n● statistical modeling:  regression analysis, time series analysis, bayesian inference, hypothesis \ntesting, experimental design  \n● programming languages:  python, r, sql, java, scala  \n \n  \n \ndear job seeker,  \n \nif you’re looking for a classy resume template, you can’t do any better than \nthe “milano”. its simple elegance is just what you need if you want to imbue \nyour resume with a balance of professionalism and good taste — plus a \ndash of creativity.  \n \neverything on the milano is left -aligned, which helps the hiring manager \nquickly look through your resume by simply scanning down the page to \nidentify relevant experience and skills.  \n \nif you’re still struggling to write your resume, here are some free resources \nto help you put together a resume that shows employers you’re the right \nperson for the job:  \n \n·     free resume builder  \n·     how to write a resume  \n·     resume samples by industry  \n \nonce you have a great resume, pair it with a convincing cover letter using \nour matching  2022 cover letter template . here are a few resources to help \nyou write a cover letter that gives your application the boost it needs to \nland you an interview:  \n·     cover letter builder  \n·     how to write a cover letter  \n·     cover letter examples by industry  \n \n \n \nbest regards,  \n \n \n \n \n \n \n \nimportant:  to delete the second page, right -click on the page and click “delete rows”
"""

In [20]:
job_description_content="""
what do we do? \n\nwe are the simulation and analytics team within volvo group trucks technology, a dynamic team working towards enabling the technology organization in evaluating futuristic products and services for volvo business units such as trucks, buses, construction equipment and marine\n\nthe main tasks in our team are to meet our customer’s expectation on quality, fuel consumption and to comply with emission legislation. this is done by selecting engine hw-components and calibrate engine related functions. upcoming emission legislation requires development of advanced control strategies together with an optimized hardware and software. you will be an important contributor in this development. powertrain is the heart of the truck and is one of the focus areas within volvo group, which is at the forefront of sustainable transport solutions for tomorrow.\n\nwhat will you do? \nyou will work in agile teams through good collaboration with our colleagues in software development and design teams all around the world.\n\nyou along with the team will be responsible for understanding customer usage and performance of our products and for providing our solutions throughout entire product life cycle, from idea investigation and concept evaluation to industrialization and to aftermarket and maintenance. you will make meaningful interpretations, recommendations and eventually predictions from the data available from various sources to support our endeavor in moving towards data driven powertrain development\n\nyou get the opportunity to follow your data driven models from script to test cell to verification in a truck and eventually to being used by our end customers.\n\nwe have an agile way of working, where each team plan their activities in sprints and deliver solutions together as a team. we strive to have an open and honest environment within the teams, where it is easy to ask each other for support when needed. the tasks can be either part of a larger project or short tasks to improve products currently in production.\n\nyou will get the opportunity to interact with highly committed colleagues from different cultures. we hope you will learn as much from us as we will from you.\n\n who are you? \n\nwe believe that to be successful in this position, you are a team player, have strong experience in data engineering and analysis area, and a will to deliver. you have a knowledge of control systems and feedback systems (closed loop) in general with an exposure to numerical and data driven simulation of a system. you must have a proven experience in data modelling – regression, clustering, neural networks, time series etc. and should have used them in solving real-life challenges (prediction, automation, real time optimization etc).\n\nyou have a willingness to learn and take more responsibility with can-do attitude.\n\nyou will be greatly appreciated in this role if you have demonstrated predictive analysis and decision-making using data.\n\nif you are a master’s degree holder in mechanical/automobile/electronics/mechatronics engineering with fantastic analytical skills, have gained a strong domain understanding in powertrain engineering with proven skills in handling and analysing large set of data to make meaningful interpretations and if you believe that you can work smoothly with python ( including libraries like numpy, scipy, pandas, tensorflow) , r, sql, git, azure, hadoop and matlab/simulink, pyspark, c/c++/ scala then you can be a good fit into this role.\n\nexperience of working with relational databases, data privacy and understanding of iot based instrumentation design with additional data logging to build or validate models is a big plus.\n\na passion for turning data into knowledge with great visualizations using power bi, qlikview, tableau and an experience of working with plant/ component models, and integration of these models into sil/mil/hil evaluations would be an icing on the cake.
"""

### 1. Resume Analysis

In [21]:
prompt_input = resume_analysis_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

**Candidate Summary:**

**Name:** Senior Data Scientist  
**Location:** Chicago, IL  
**Contact:** your.email@email.com | (312) 875-3382  

**Experience:**
- **Total Experience:** Over 10 years in data science and analytics.
- **Current Position:** Senior Data Scientist at Hargrove Tech (August 20xx – Present)
  - Leads a team of 8 data scientists.
  - Developed advanced machine learning models, achieving a 25% increase in predictive accuracy.
  - Initiated a company-wide optimization of data pipelines, reducing processing time by 40% and saving $1.2 million annually.
  - Collaborates with cross-functional teams to enhance data-driven strategies, contributing to a 15% revenue increase.

- **Previous Position:** Lead Data Scientist at SunTrust Analytics (July 20xx – August 20xx)
  - Developed a real-time fraud detection system using ensemble learning, saving $3.5 million in potential losses.
  - Created a customer segmentation model that improved targeted marketing ROI by 20%.
  - Condu

### 2. Job Description Analysis

In [12]:
prompt_input = job_description_analysis_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 Thank you for providing the job description. Based on the requirements listed, the key skills, qualifications, and experience needed for this position are: 
- 10+ years of experience as a data scientist 
- Expertise in machine learning, statistical modeling, and big data analytics 
- Proven track record of leading cross-functional teams 
- Experience with developing and implementing data-driven strategies 
- Strong communication and collaboration skills 
- Proficiency in programming languages such as Python, R, SQL, Java, and Scala 
- Master's degree in Data Science or a related field 

In order to align your resume with this job description, here are some suggestions: 

- Highlight your 10+ years of experience as a data scientist in your professional experience section. 
- Emphasize your expertise in machine learning, statistical modeling, and big data analytics by providing specific examples of projects you have worked on and the results you have achieved. 
- Showcase your leadershi

### 3. Resume Comparison with JD

In [13]:
prompt_input = comparison_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 Hello! Thank you for submitting your resume for review. After reviewing your resume and the job description, I have identified a few areas where your resume could be improved to better align with the job requirements. Firstly, the job description mentions a focus on sustainable transport solutions, but your resume does not mention any experience or skills in this area. It would be beneficial to highlight any relevant experience or knowledge you have in this field. Additionally, the job description mentions a need for experience in data engineering and analysis, but your resume does not mention any specific experience or skills in this area. It would be helpful to provide more details on your experience with data modelling and analysis, as well as any specific tools or languages you are proficient in. Finally, the job description mentions a preference for candidates with a master's degree in a relevant field, but your resume does not mention your educational background. It would be ben

### 4. Gap Analysis wrt JD

In [15]:
prompt_input = gap_analysis_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 After reviewing the resume and job description, I have identified a few areas where the skills and experience detailed in the resume do not align with the requirements listed in the job description. These gaps can be addressed by making some changes to the resume and highlighting certain skills and experiences.

1. The job description mentions a need for experience in control systems and feedback systems, but the resume does not mention any specific experience in this area. To fulfill this gap, the resume should highlight any relevant experience or knowledge in control systems and feedback systems, such as experience with closed loop systems or coursework in this area.

2. The job description also mentions a need for experience in data modelling, specifically with regression, clustering, neural networks, and time series. While the resume mentions experience with machine learning and statistical modeling, it does not specify experience with these specific techniques. To fulfill this ga

### 5. How to improve the experience?

In [14]:
prompt_input = experience_enhancement_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 Hello, thank you for submitting your resume for review. Based on your experience, I can see that you have a strong background in data science and analytics. To further improve your qualifications for this role, I would recommend gaining experience in powertrain engineering and control systems. This can be achieved through additional courses or certifications in these areas. Additionally, having knowledge of relational databases and data privacy would also be beneficial. I would also suggest gaining experience with data visualization tools such as Power BI, QlikView, or Tableau. This will help you effectively communicate your findings and insights to others. Overall, your experience and skills make you a strong candidate for this role, and I would encourage you to continue developing your skills in these areas.


### 6. Resume Reformatting

In [15]:
prompt_input = resume_formatting_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 Hello, thank you for submitting your resume for review. I have some suggestions on how you can enhance the formatting to make it more visually appealing and easier to read. First, I recommend using a professional resume template, such as the "Milano" template, to give your resume a balance of professionalism and creativity. Additionally, I suggest left-aligning all the content to make it easier for the hiring manager to scan through your experience and skills. 

As for the content, I see that you have a strong background in data science and have achieved impressive results in your previous roles. However, I suggest organizing your experience in bullet points to make it more concise and easier to read. Also, consider using action verbs to highlight your accomplishments, such as "led a team of 8 data scientists" and "spearheaded a company-wide initiative."

In terms of your education and skills, I recommend placing them at the top of your resume to make them more prominent. This will al

### 7. Resume Length Analysis

In [16]:
prompt_input = resume_length_prompt.format(resume_content, job_description_content)
output = resume_chain.predict(input=prompt_input)
print(output)

 Hello! Thank you for submitting your resume for review. Based on the job description, it seems like you have a strong background in data science and analytics. However, I noticed that your resume is quite long and may benefit from some adjustments to make it more concise and aligned with the job requirements. 

Firstly, I suggest focusing on highlighting your experience and skills in data engineering and analysis, as well as your knowledge of control systems and feedback systems. These are important areas for this role and should be emphasized in your resume. 

Additionally, it may be helpful to provide specific examples of how you have used data modelling techniques such as regression, clustering, and neural networks to solve real-life challenges. This will demonstrate your ability to apply these skills in a practical setting. 

Furthermore, I recommend removing any irrelevant information or experiences that do not directly relate to the job requirements. This will help keep your res